# Installation of dependencies

In [1]:
pip install gemmi wget matplotlib tqdm ipywidgets

  Using cached wget-3.2.zip (10 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.7 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9685 sha256=58e1314740b089a90c3cca3886ba73c1811a420faf7443e17c34c0aff6adda9c
  Stored in directory: /Users/jon/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget

# PDB mirror update (structures only)

In [9]:
%%sh

MIRRORDIR=/Users/jon/repos/mmCIF_PDB
LOGFILE=/Users/jon/repos/mmCIF_PDB_`date +"%Y-%m-%d"`.log
RSYNC=rsync
echo "Log file: " $LOGFILE
SERVER=rsync.ebi.ac.uk::pub/databases/pdb
PORT=873

mkdir -p $MIRRORDIR

${RSYNC} -rlpt -v -z --delete --port=$PORT ${SERVER}/data/structures/divided/mmCIF/ $MIRRORDIR > $LOGFILE 2>/dev/null

Log file:  /Users/jon/repos/mmCIF_PDB_2025-06-20.log


# Reading all structures

In [ ]:
import gemmi
from tqdm.notebook import tqdm

loaded_count = 0
failed_count = 0

for file_path in tqdm(gemmi.CoorFileWalk("/Users/jon/repos/mmCIF_PDB")) :
  try:
    structure = gemmi.read_structure(file_path)
    print(f"Loaded: {file_path} | Structure name: {structure.name}")
    loaded_count += 1
  except Exception as e:
    print(f"Failed: {file_path} | Error: {e}")
    failed_count += 1

print(f"Summary: Loaded {loaded_count} files, Failed {failed_count} files")
